In [1]:
from enigma.enigma import Enigma
from enigma.rotors.rotor_with_mapping_and_notches import RotorWithMappingAndNotches
from enigma.rotors.rotor_I import RotorI
from enigma.rotors.rotor_II import RotorII
from enigma.rotors.rotor_III import RotorIII
from enigma.rotors.rotor_IV import RotorIV
from enigma.rotors.rotor_V import RotorV
from enigma.reflectors.reflector_b import ReflectorB
from enigma.plugboard import Plugboard
from language_models.character_frequency_kld_language_model import CharacterFrequencyKLDLanguageModel
from language_models.markov_chain_model import MarkovChainModel

import timeit
import random
from collections import defaultdict
import heapq
from itertools import permutations, combinations
from tqdm import tqdm
from tqdm.auto import trange
from multiprocessing import Pool
from functools import partial
import matplotlib.pyplot as plt

In [2]:
POSSIBLE_ROTORS = {RotorI, RotorII, RotorIII, RotorIV, RotorV}
POSSIBLE_LETTERS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def get_random_plugboad(plugboard_size: int):
    letter_sample = random.sample(list(POSSIBLE_LETTERS), plugboard_size*2)
    plugboard_tuples = []
    for i in range(len(letter_sample)//2):
        plugboard_tuples.append((letter_sample[i*2], letter_sample[(i*2)+1]))
    plugboard_tuples = sorted(plugboard_tuples, key=lambda x: x[0])
    return plugboard_tuples

def get_random_config(plugboard_size: int):
    rotors_to_use = random.sample(POSSIBLE_ROTORS, 3)
    offsets = [random.randint(0,25) for _ in range(3)]
    plugboard_tuples = get_random_plugboad(plugboard_size)
    return rotors_to_use, offsets, plugboard_tuples

def string_to_config(string: str):
    match = re.findall("(.+)\(o:(\d+),rs:(\d+)\)\|(.+)\(o:(\d+),rs:(\d+)\)\|(.+)\(o:(\d+),rs:(\d+)\)\|(.*)", 
                       string, re.IGNORECASE)[0]
    available_rotors = {rot.__name__:rot for rot in RotorWithMappingAndNotches.__subclasses__()}
    rotors_classes = [available_rotors[match[0]], available_rotors[match[3]], available_rotors[match[6]]]
    offsets = [int(match[1]), int(match[4]), int(match[7])]
    ringstellungs = [int(match[2]), int(match[5]), int(match[8])]
    plugboard_tuples = []
    for i in range(len(match[9])//2):
        plugboard_tuples.append((match[9][i*2],match[9][(i*2)+1]))
    return rotors_classes, offsets, ringstellungs, plugboard_tuples

def config_to_string(rotors_classes, offsets, ringstellungs, plugboard_tuples):
    string = ""
    for rot, off, rs in zip(rotors_classes, offsets, ringstellungs):
        string += rot.__name__+"(o:%d,rs:%d)"%(off,rs)+"|"
    for l1, l2 in plugboard_tuples:
        string += l1+l2
    return string

def transform_to_valid_chars(text: str):
    text = text.upper()
    return "".join([c for c in text if c in POSSIBLE_LETTERS])

In [3]:
# Me defino una enigma
PLUGBOARD_SIZE = 10 #Size used by Germany in WW2
rotors, offsets, plugboard_tuples = get_random_config(PLUGBOARD_SIZE)
rotors_to_use = [rot_c(offset=off) 
                 for rot_c, off in zip(rotors, offsets)]
enigma = Enigma(reflector=ReflectorB(),
                        plugboard=Plugboard(plugboard_tuples),
                        rotors=rotors_to_use)

In [10]:
# Modelo de lenguaje basado en la frecuencia de caracteres en un libro en ingles
with open('books/Alices_Adventures_in_Wonderland.txt', 'r') as book:
    train_book = book.read()
freq_kld_model = CharacterFrequencyKLDLanguageModel(train_book)
markov_1_model = MarkovChainModel(train_book)

with open('books/Frankenstein.txt', 'r') as book:
    frankestein_book = transform_to_valid_chars("".join(book.readlines()))

In [14]:
# Me genero todas las posibles combinaciones de los offsets de los rotores, para dos modelos distintos

def crack(encrypted_message):
    kb_divs, markov_1_divs = {}, {}
    
    #for offsets in tqdm(range(26**3), total=26**3):
    for offsets in range(26**3):
        off1 = offsets % 26
        off2 = (offsets//26) % 26
        off3 = (offsets//(26**2)) % 26
        enigma_with_given_offset = Enigma(ReflectorB(),
                                          plugboard=Plugboard(plugboard_tuples),
                                          rotors= [rot_c(offset=off) 
                                                   for rot_c, off in zip(rotors, [off1, off2, off3])]
                                         )
        
        decrypted_message = enigma_with_given_offset.decrypt(encrypted_message)
        kb_divs[(off1, off2, off3)] = freq_kld_model.fitness(decrypted_message)
        markov_1_divs[(off1, off2, off3)] = markov_1_model.fitness(decrypted_message)
    
    kb_divs = sorted(kb_divs.items(), key=lambda x: x[1], reverse = True)
    markov_1_divs = sorted(markov_1_divs.items(), key=lambda x: x[1], reverse = True)
  
    return kb_divs, markov_1_divs

In [ ]:
%%time
# Para cada longitud, pruebo los diferentes modelos para los diferentes tipos de rotores

MIN_LEN, MAX_LEN, STEP_LEN = 5, 255, 5
NUMBER_OF_SAMPLES = 10

crack_time_given_length = defaultdict(list)
rankings_freq, rankings_markov_1 = defaultdict(list), defaultdict(list)

#for length in tqdm(range(MIN_LEN, MAX_LEN, STEP_LEN)):
for length in tqdm(range(100, 101)):
    for _ in tqdm(range(NUMBER_OF_SAMPLES)):
        import timeit
        
        start = random.randint(0, len(frankestein_book)-length) #Para que si o si tenga lenght 
        encrypted_message = enigma.encrypt(frankestein_book[start: start + length])
        start_time_of_crack = timeit.default_timer()
        kb_divs, markov_1_divs = crack(encrypted_message)
        kb_divs_rots, markov_1_rots = [x[0] for x in kb_divs], [x[0] for x in markov_1_divs]
        
        crack_time_given_length[length].append(timeit.default_timer() - start_time_of_crack)
        rankings_freq[length].append(kb_divs_rots.index(tuple(offsets)))
        rankings_markov_1[length].append(markov_1_rots.index(tuple(offsets)))


 30%|███       | 3/10 [01:31<03:29, 29.98s/it]

In [66]:
rankings_freq, rankings_markov_1, crack_time_given_length

(defaultdict(list, {100: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}),
 defaultdict(list,
             {100: [4816,
               4816,
               4816,
               4816,
               4816,
               4816,
               4816,
               4816,
               4816,
               4816]}),
 defaultdict(list,
             {100: [11.64096172000427,
               11.760646540999005,
               11.541344733006554,
               11.554207290995691,
               11.571838194999145,
               11.583171446000051,
               11.670931143002235,
               11.575419018001412,
               11.63612681599625,
               11.65871626700391]}))